<a href="https://colab.research.google.com/github/lyk1231/notes/blob/main/practice_jobs_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import report of jobs
#clean and transform

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import datetime

In [ ]:
#import data
sheet_id = '1lA-5IKOd-0DYwI93PP5ciRrX92pjgX7RrjFwSAomHfw'
sheet_name = 'Sheet1'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url)

In [ ]:
#import another sheet with lookup references
sheet_name_glossary = 'glossary'
url_glossary = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_glossary}'
df_glossary = pd.read_csv(url_glossary)

In [ ]:
#correct data type
df['start date'] = pd.to_datetime(df['start date'])
# df.pop('days open')

#include days open col
df.insert(df.columns.get_loc('start date')+1, 'days open', pd.to_datetime(date.today()) - df['start date'])
df['days open'] = df['days open'].dt.days

print(df.dtypes)

#reorganize column order
tempCol = df['location']
df.pop('location')
df.insert(df.columns.get_loc('division')+1, tempCol.name, tempCol)

job title                  object
openings                    int64
recruiter                  object
location                   object
hiring manager             object
start date         datetime64[ns]
days open                   int64
reason for hire            object
replacement for            object
department                 object
business unit             float64
division                  float64
dtype: object


In [ ]:
#credit: https://pythoninoffice.com/replicate-excel-vlookup-hlookup-xlookup-in-python/
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [ ]:
df.isna().sum()

# fill in bu, division
# xlookup('whoever1', df['replacement for'], df['department'])
df['business unit'] = df['department'].apply(xlookup, args = (df_glossary['department'], df_glossary['business unit']))
df['division'] = df['department'].apply(xlookup, args = (df_glossary['department'], df_glossary['division']))

In [ ]:
print(df.isna().sum())

df[df['replacement for'].isna()]

job title          0
openings           0
recruiter          0
hiring manager     0
start date         0
days open          0
reason for hire    0
replacement for    4
department         0
business unit      0
division           0
location           0
dtype: int64


,job title,openings,recruiter,hiring manager,start date,days open,reason for hire,replacement for,department,business unit,division,location
7,h,1,symone d,schmitt,2022-05-22,41,recruiting > newfill,NaN,software developer,engineering,science,remote nj
8,i,1,alex m,schmitt,2022-02-19,133,NEW POSITION,NaN,software developer,engineering,science,remote ny
9,j,5,denny s,adams,2022-01-25,158,NEW POSITION,NaN,design,marketing,creative,remote ny
10,k,1,symone d,schmitt,2022-06-29,3,NEW POSITION,NaN,software developer,engineering,science,remote ny


In [ ]:
#sort
df.sort_values(['recruiter', 'job title', 'department'], inplace=True)

In [25]:
#categorize reqs by days open
df['days open category'].mask((df['days open'] > 0) & (df['days open'] < 30), 'Green', inplace=True)
df['days open category'].mask((df['days open'] > 30) & (df['days open'] < 45), 'Yellow', inplace=True)
df['days open category'].mask(df['days open'] > 45, 'Red', inplace=True)

tempCol = df['days open category']
df.pop('days open category')
df.insert(df.columns.get_loc('days open')+1, 'days open category', tempCol)

In [23]:
#correct reason for hire text
df['reason for hire'].mask(df['reason for hire'] == 'CURR POSITION', 'recruiting > backfill', inplace=True)
df['reason for hire'].mask(df['reason for hire'] == 'NEW POSITION', 'recruiting > new hire', inplace=True)

In [55]:
#average days open
df['days open'].sum()/ df['days open'].count()

df.groupby('recruiter').sum()['openings'].reset_index().sort_values('openings', ascending=False)

,recruiter,openings
1,denny s,8
2,symone d,5
0,alex m,4
